## Setup: Load Example Data and Compute Metrics

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
from dosemetrics import read_dose_and_mask_files, compute_dvh
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Download dataset
data_path = Path(snapshot_download(
    repo_id="contouraid/dosemetrics-data",
    repo_type="dataset"
))

# Load test subject
subject_path = data_path / "longitudinal" / "time_point_1"
dose, structures = read_dose_and_mask_files(subject_path)

# Create output directory
output_dir = Path("dosemetrics_output")
output_dir.mkdir(exist_ok=True)

print(f"✓ Data loaded")
print(f"✓ Output directory: {output_dir}")

## 1. Exporting DVH Data to CSV

Export raw DVH data for further analysis in spreadsheet software.

In [ ]:
# Compute DVHs for all structures
dvh_data = {}
for structure_name in structures.structure_names:
    mask = structures.get_structure_mask(structure_name)
    dvh = compute_dvh(dose, mask, organ_name=structure_name)
    dvh_data[structure_name] = dvh

# Create DataFrame with DVH curves
# Use dose bins as index
dose_bins = dvh_data["PTV"].dose_bins
dvh_df = pd.DataFrame({"Dose_Gy": dose_bins})

for structure_name, dvh in dvh_data.items():
    dvh_df[f"{structure_name}_Volume_%"] = dvh.volume_fractions * 100

# Export to CSV
csv_file = output_dir / "dvh_curves.csv"
dvh_df.to_csv(csv_file, index=False, float_format='%.3f')
print(f"✓ DVH curves exported to: {csv_file}")
print(f"  Shape: {dvh_df.shape}")
print(f"\nFirst few rows:")
print(dvh_df.head())

## 2. Exporting Summary Statistics

Create a summary table with key dosimetric parameters.

In [ ]:
# Create summary statistics table
summary_data = []

for structure_name, dvh in dvh_data.items():
    summary_data.append({
        'Structure': structure_name,
        'Mean_Dose_Gy': dvh.mean_dose,
        'Max_Dose_Gy': dvh.max_dose,
        'Min_Dose_Gy': dvh.min_dose,
        'D98_Gy': dvh.get_dose_at_volume(98),
        'D95_Gy': dvh.get_dose_at_volume(95),
        'D50_Gy': dvh.get_dose_at_volume(50),
        'D2_Gy': dvh.get_dose_at_volume(2),
    })

summary_df = pd.DataFrame(summary_data)

# Export to CSV
summary_csv = output_dir / "dose_statistics.csv"
summary_df.to_csv(summary_csv, index=False, float_format='%.2f')
print(f"✓ Dose statistics exported to: {summary_csv}")
print(f"\nSummary statistics:")
print(summary_df.to_string(index=False))

## 3. Exporting to Excel with Multiple Sheets

Combine multiple tables into a single Excel file.

In [ ]:
# Create Excel file with multiple sheets
excel_file = output_dir / "dosimetric_analysis.xlsx"

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    # Sheet 1: Summary statistics
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Sheet 2: DVH curves
    dvh_df.to_excel(writer, sheet_name='DVH_Curves', index=False)
    
    # Sheet 3: Constraint compliance
    constraints_data = []
    constraint_limits = {
        'PTV': [('D95', 95, '>=', 50.0)],
        'BrainStem': [('Max', 100, '<=', 54.0)],
        'Chiasm': [('Max', 100, '<=', 54.0)],
        'Eye_L': [('Max', 100, '<=', 45.0)],
        'Eye_R': [('Max', 100, '<=', 45.0)],
    }
    
    for structure_name, constraints in constraint_limits.items():
        if structure_name in dvh_data:
            dvh = dvh_data[structure_name]
            for metric_name, volume_pct, operator, limit in constraints:
                if metric_name == 'Max':
                    value = dvh.max_dose
                else:
                    value = dvh.get_dose_at_volume(volume_pct)
                
                if operator == '>=':
                    passed = value >= limit
                else:  # '<='
                    passed = value <= limit
                
                constraints_data.append({
                    'Structure': structure_name,
                    'Constraint': f"{metric_name} {operator} {limit} Gy",
                    'Value_Gy': value,
                    'Limit_Gy': limit,
                    'Pass': 'Yes' if passed else 'No'
                })
    
    constraints_df = pd.DataFrame(constraints_data)
    constraints_df.to_excel(writer, sheet_name='Constraints', index=False)

print(f"✓ Excel file exported to: {excel_file}")
print(f"  Contains 3 sheets: Summary, DVH_Curves, Constraints")

## 4. Creating Publication-Quality DVH Plots

Generate high-resolution plots suitable for publications.

In [ ]:
# Configure matplotlib for publication quality
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['lines.linewidth'] = 2

# Create figure
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)

# Define colors and line styles
structure_styles = {
    'PTV': {'color': '#E74C3C', 'linestyle': '-', 'linewidth': 2.5},
    'Brain': {'color': '#3498DB', 'linestyle': '--', 'linewidth': 2},
    'BrainStem': {'color': '#E67E22', 'linestyle': '-', 'linewidth': 2},
    'Chiasm': {'color': '#2ECC71', 'linestyle': '-', 'linewidth': 2},
    'Eye_L': {'color': '#9B59B6', 'linestyle': ':', 'linewidth': 2},
    'Eye_R': {'color': '#9B59B6', 'linestyle': '-.', 'linewidth': 2},
}

# Plot DVHs
for structure_name, style in structure_styles.items():
    if structure_name in dvh_data:
        dvh = dvh_data[structure_name]
        ax.plot(dvh.dose_bins, dvh.volume_fractions * 100, 
               label=structure_name, **style)

# Formatting
ax.set_xlabel('Dose (Gy)', fontsize=13, fontweight='bold')
ax.set_ylabel('Volume (%)', fontsize=13, fontweight='bold')
ax.set_title('Dose-Volume Histogram', fontsize=15, fontweight='bold', pad=15)
ax.legend(loc='best', framealpha=0.95, fontsize=10, edgecolor='black')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim(0, None)
ax.set_ylim(0, 105)

# Add margin
plt.tight_layout()

# Save in multiple formats
for fmt in ['png', 'pdf', 'svg']:
    output_file = output_dir / f"dvh_plot.{fmt}"
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"✓ Plot saved: {output_file}")

plt.show()

## 5. Creating Comparison Plots

Compare two treatment plans side by side.

In [ ]:
# Load comparison plans
plan1_path = data_path / "longitudinal" / "time_point_1"
plan2_path = data_path / "longitudinal" / "time_point_2"

dose1, structures1 = read_dose_and_mask_files(plan1_path)
dose2, structures2 = read_dose_and_mask_files(plan2_path)

# Create side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), dpi=300)

compare_structures = ['Brainstem', 'Chiasm', 'Eye_L', 'Eye_R']
colors = ['#E67E22', '#2ECC71', '#9B59B6', '#8E44AD']

# Plot Plan 1
for i, struct_name in enumerate(compare_structures):
    if structures1.has_structure(struct_name):
        mask = structures1.get_structure_mask(struct_name)
        dvh = compute_dvh(dose1, mask, organ_name=struct_name)
        ax1.plot(dvh.dose_bins, dvh.volume_fractions * 100, 
                label=struct_name, color=colors[i], linewidth=2)

ax1.set_xlabel('Dose (Gy)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Volume (%)', fontsize=12, fontweight='bold')
ax1.set_title('Plan 1 (First)', fontsize=13, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 70)
ax1.set_ylim(0, 105)

# Plot Plan 2
for i, struct_name in enumerate(compare_structures):
    if structures2.has_structure(struct_name):
        mask = structures2.get_structure_mask(struct_name)
        dvh = compute_dvh(dose2, mask, organ_name=struct_name)
        ax2.plot(dvh.dose_bins, dvh.volume_fractions * 100, 
                label=struct_name, color=colors[i], linewidth=2)

ax2.set_xlabel('Dose (Gy)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Volume (%)', fontsize=12, fontweight='bold')
ax2.set_title('Plan 2 (Last)', fontsize=13, fontweight='bold')
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 70)
ax2.set_ylim(0, 105)

plt.tight_layout()

# Save
comparison_file = output_dir / "plan_comparison.png"
plt.savefig(comparison_file, dpi=300, bbox_inches='tight')
print(f"✓ Comparison plot saved: {comparison_file}")

plt.show()

## 6. Creating a Comprehensive HTML Report

Generate a complete HTML report with all analysis results.

In [ ]:
from datetime import datetime

# Create HTML report
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Dosimetric Analysis Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; background-color: #f5f5f5; }}
        .container {{ background-color: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
        h1 {{ color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }}
        h2 {{ color: #34495e; margin-top: 30px; }}
        table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
        th {{ background-color: #3498db; color: white; font-weight: bold; }}
        tr:nth-child(even) {{ background-color: #f2f2f2; }}
        .pass {{ color: green; font-weight: bold; }}
        .fail {{ color: red; font-weight: bold; }}
        .info {{ background-color: #e8f4f8; padding: 15px; border-left: 4px solid #3498db; margin: 20px 0; }}
        img {{ max-width: 100%; height: auto; margin: 20px 0; border: 1px solid #ddd; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Dosimetric Analysis Report</h1>
        
        <div class="info">
            <strong>Subject:</strong> test_subject<br>
            <strong>Analysis Date:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}<br>
            <strong>Software:</strong> DoseMetrics<br>
            <strong>Number of Structures:</strong> {len(structures.structure_names)}
        </div>
        
        <h2>Dose Statistics Summary</h2>
        {summary_df.to_html(index=False, float_format='%.2f')}
        
        <h2>Dose-Volume Histogram</h2>
        <img src="dvh_plot.png" alt="DVH Plot">
        
        <h2>Constraint Compliance</h2>
        {constraints_df.to_html(index=False, float_format='%.2f')}
        
        <h2>Analysis Files</h2>
        <ul>
            <li>DVH Curves: <code>dvh_curves.csv</code></li>
            <li>Dose Statistics: <code>dose_statistics.csv</code></li>
            <li>Complete Analysis: <code>dosimetric_analysis.xlsx</code></li>
            <li>DVH Plot: <code>dvh_plot.png</code>, <code>dvh_plot.pdf</code>, <code>dvh_plot.svg</code></li>
        </ul>
        
        <hr>
        <p style="color: #7f8c8d; font-size: 0.9em;">
            Generated by DoseMetrics | 
            <a href="https://github.com/contouraid/dosemetrics">GitHub</a> | 
            <a href="https://contouraid.github.io/dosemetrics/">Documentation</a>
        </p>
    </div>
</body>
</html>
"""

# Save HTML report
html_file = output_dir / "analysis_report.html"
with open(html_file, 'w') as f:
    f.write(html_content)

print(f"✓ HTML report generated: {html_file}")
print(f"  Open in browser to view the complete report")

## 7. Batch Export for Multiple Subjects

Process and export results for multiple subjects at once.

In [ ]:
# Find all subject folders
subject_folders = [d for d in data_path.iterdir() 
                  if d.is_dir() and not d.name.startswith('.')]

print(f"Found {len(subject_folders)} subjects to process")

# Process each subject
batch_results = []

for subject_folder in subject_folders:
    print(f"\nProcessing: {subject_folder.name}")
    
    try:
        # Load data
        dose, structures = read_dose_and_mask_files(subject_folder)
        
        # Compute metrics for key structures
        for structure_name in ['PTV', 'Brain', 'BrainStem', 'Brainstem']:
            if structures.has_structure(structure_name):
                mask = structures.get_structure_mask(structure_name)
                dvh = compute_dvh(dose, mask, organ_name=structure_name)
                
                batch_results.append({
                    'Subject': subject_folder.name,
                    'Structure': structure_name,
                    'Mean_Dose_Gy': dvh.mean_dose,
                    'Max_Dose_Gy': dvh.max_dose,
                    'D95_Gy': dvh.get_dose_at_volume(95),
                    'D50_Gy': dvh.get_dose_at_volume(50),
                })
        
        print(f"  ✓ Completed")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")

# Create batch results DataFrame
batch_df = pd.DataFrame(batch_results)

# Export batch results
batch_csv = output_dir / "batch_analysis.csv"
batch_df.to_csv(batch_csv, index=False, float_format='%.2f')
print(f"\n✓ Batch results exported to: {batch_csv}")
print(f"  Total rows: {len(batch_df)}")
print(f"\nBatch results preview:")
print(batch_df.head(10))

## 8. Summary of Exported Files

Review all files created during this analysis.

In [ ]:
import os

print("Exported Files Summary:")
print("=" * 70)

for file in sorted(output_dir.iterdir()):
    if file.is_file():
        size_kb = file.stat().st_size / 1024
        print(f"{file.name:40s} {size_kb:8.1f} KB")

print("=" * 70)
total_size = sum(f.stat().st_size for f in output_dir.iterdir() if f.is_file()) / 1024
print(f"{'Total':40s} {total_size:8.1f} KB")

print(f"\n✓ All exports complete!")
print(f"\nOutput directory: {output_dir.absolute()}")

## Summary

In this notebook, you learned how to:

1. ✓ Export DVH data to CSV
2. ✓ Create summary statistics tables
3. ✓ Export to Excel with multiple sheets
4. ✓ Create publication-quality plots (PNG, PDF, SVG)
5. ✓ Generate comparison plots
6. ✓ Create comprehensive HTML reports
7. ✓ Perform batch processing and export
8. ✓ Manage output files

## Export Formats Summary

| Format | Use Case | Files |
|--------|----------|-------|
| **CSV** | Spreadsheet analysis, data sharing | `dvh_curves.csv`, `dose_statistics.csv` |
| **Excel** | Comprehensive analysis in one file | `dosimetric_analysis.xlsx` |
| **PNG** | Presentations, web display | `dvh_plot.png` |
| **PDF** | Publications, reports | `dvh_plot.pdf` |
| **SVG** | Scalable graphics, editing | `dvh_plot.svg` |
| **HTML** | Interactive reports | `analysis_report.html` |

## Next Steps

- **Automation**: Create scripts to run batch analysis automatically
- **Integration**: Incorporate exports into clinical workflows
- **Custom Reports**: Modify HTML templates for specific needs

## References

- [DoseMetrics Documentation](https://contouraid.github.io/dosemetrics/)
- [Batch Processing Guide](https://contouraid.github.io/dosemetrics/user-guide/batch-processing/)
- [GitHub Repository](https://github.com/contouraid/dosemetrics)